# Demo
1. Show two json files as the inputs
2. Post to terrarium funman api, show response id
3. Get result from id, and plot

# Todo
1. How to interact with terrarium
2. Setup API for two json files
3. Setup async anytime results
4. Truncate/halt request endpoint
5. Allow multiple requests in a queue, needs server thread

In [ ]:
import asyncio
import nest_asyncio
import json
import unittest
from os import path, getcwd

import funman.api.client as client
from funman.api.api import app
from funman.api.server import Server, ServerConfig

API_BASE_PATH = path.join(getcwd(), "..")
API_SERVER_HOST = "0.0.0.0"
API_SERVER_PORT = 8190
SERVER_URL = f"http://{API_SERVER_HOST}:{API_SERVER_PORT}"
OPENAPI_URL = f"{SERVER_URL}/openapi.json"
CLIENT_NAME = "funman-api-client"

In [ ]:
# Start API Server
server = Server(
            config=ServerConfig(
                app,
                host=API_SERVER_HOST,
                port=API_SERVER_PORT,
                log_level="info",
            )
        )
nest_asyncio.apply()
thread = server.start_in_thread()

In [ ]:
# Install API client to the working directory
# and add the install to the sys path
client.make_client(
    API_BASE_PATH, openapi_url=OPENAPI_URL, client_name=CLIENT_NAME
)

In [ ]:
# Setup API client request
from funman_api_client import Client
from funman_api_client.api.default import (
    post_queries_queries_post,
)
from funman_api_client.models.body_post_queries_queries_post import BodyPostQueriesQueriesPost
from funman_api_client.models import (
    BilayerModel,
    FUNMANConfig,
    LabeledParameter,
    QueryLE,
    QueryRequest,
    QueryResponse
)

funman_client = Client(SERVER_URL, timeout=None)

In [ ]:
bilayer_json =     {"Wa":[{"influx":1,"infusion":2},
			 {"influx":2,"infusion":3}],
 "Win":[{"arg":1,"call":1},
 				{"arg":2,"call":1},
 				{"arg":2,"call":2}],
 "Box":[{"parameter":"beta"},
 				{"parameter":"gamma"}],
 "Qin":[{"variable":"S"},
 				{"variable":"I"},
 				{"variable":"R"}],
 "Qout":[{"tanvar":"S'"},
 				 {"tanvar":"I'"},
 				 {"tanvar":"R'"}],
 "Wn":[{"efflux":1,"effusion":1},
 			 {"efflux":2,"effusion":2}]}

infected_threshold = 130
init_values = {"S": 9998, "I": 1, "R": 1}

lb = 0.000067 * (1 - 0.5)
ub = 0.000067 * (1 + 0.5)

response = asyncio.run(
    post_queries_queries_post.asyncio_detailed(
        client=funman_client,
        json_body=BodyPostQueriesQueriesPost(
            QueryRequest(
                model = BilayerModel.from_dict(
                    {
                        "init_values": init_values,
                        "bilayer": {"json_graph": bilayer_json},
                    }
                ),
                query = QueryLE.from_dict(
                    {
                        "variable": "I",
                        "ub": infected_threshold,
                        "at_end": False,
                    }
                ),
                parameters = [
                    #LabeledParameter.from_dict({
                    #    "name": "beta",
                    #    "lb": lb,      
                    #    "ub": ub,
                    #    "label": "all",
                    #})
                ]
            ),
            FUNMANConfig.from_dict(
                {
                    "tolerance": 1.0e-8,
                    "number_of_processes": 1
                }
            ),  
        ),
    )
)

result = QueryResponse.from_dict(
    src_dict=json.loads(response.content.decode())
)
result.parameter_space

In [ ]:
from funman_demo import plot
plot.plot_parameter_space(result.parameter_space, 1.0, True)

In [ ]:
# Close out server
server.should_exit = True
thread.join()